In [1]:
// # SUDOKU Solver
// ---- Board Setup ----
// A sudoku puzzle consists of a 9x9 grid of boxes, filled with digits from 1 - 9.

// We label 3 distinct entities in the board.

// boxes - the individual boxes in the board(9x9=81)
// units - rows, columns and the subsquares(3 x 9=27)
// peers - a box has peers in a row a column and a subsquare(8 + 6 + 6=20)

In [2]:
// ---- Encoding the board ----
// Rows will be labeled from 'A-I'
// Columns will be labeled from '1-9'

// A board will be encoded in two ways.
// 1. a long string with box contents in a series from left to right, top to bottom
// 2. a dictionary with box labels as keys, i.e. 'A1' and the box contents as values

// For both encodings, the values for empty boxes is a '.' (period).

In [3]:
let rows = 'ABCDEFGHI';
let cols = '123456789';

In [4]:
// cross() helper function creates all combinations of letters in 2 strings
//function cross(a, b) {
//  let result = [];
//  for (let r of a) {
//    for (let c of b) {
//      result.push(r + c);
//    }
//  }
//  return result;
//}

In [5]:
// This is a more concise version of the cross helper function
function cross(a, b) {
  return [...a].flatMap(r => [...b].map(c => r + c));
}

In [6]:
const boxes = cross(rows, cols);

In [7]:
const row_units = rows.split('').map(r => cross(r, cols));

In [8]:
const column_units = cols.split('').map(c => cross(rows, c));

In [9]:
const square_units = ['ABC', 'DEF', 'GHI'].flatMap(rs => ['123', '456', '789'].map(cs => cross(rs, cs)));

In [10]:
const unitlist = row_units.concat(column_units, square_units);

In [11]:
const units = {};
for (let s of boxes) {
  const x = [];
  for (let u of unitlist) {
    if (u.includes(s)) {
      x.push(u);
    }
  }
  units[s] = x;
}

[
  [
    'I1', 'I2', 'I3',
    'I4', 'I5', 'I6',
    'I7', 'I8', 'I9'
  ],
  [
    'A9', 'B9', 'C9',
    'D9', 'E9', 'F9',
    'G9', 'H9', 'I9'
  ],
  [
    'G7', 'G8', 'G9',
    'H7', 'H8', 'H9',
    'I7', 'I8', 'I9'
  ]
]

In [12]:
const peers = {};
for (let s of boxes) {
  peers[s] = new Set(
    [].concat(...units[s]).filter(item => item !== s)
  );
}

Set(20) {
  'I1',
  'I2',
  'I3',
  'I4',
  'I5',
  'I6',
  'I7',
  'I8',
  'A9',
  'B9',
  'C9',
  'D9',
  'E9',
  'F9',
  'G9',
  'H9',
  'G7',
  'G8',
  'H7',
  'H8'
}

In [13]:
// Convert string representation of puzzle into dictionary
function gridValues(grid) {
    const values = [];
    const allDigits = '123456789';

    for (let c of grid) {
        if (c === '.') {
            values.push(allDigits);
        } else if (allDigits.includes(c)) {
          values.push(c);
        }
    }
    if (values.length !== 81) {
        throw new Error('Input grid does not have 81 characters!');
    }
    const result = {};
    for (let i = 0; i < boxes.length; i++) {
        result[boxes[i]] = values[i];
    }
    return result;
}

In [14]:
// ---- Display 2D grid ----
// Input: dictionary
function displayDot(values) {
  const line = '------+------+------';
  for (let r of rows) {
    let rowOutput = '';
    for (let c of cols) {
      const value = values[r + c].length === 1 ? values[r + c] : '.';
      rowOutput += value + ' ' + (c === '3' || c === '6' ? '|' : '');
    }
    console.log(rowOutput);
    if (r === 'C' || r === 'F') {
      console.log(line);
    }
  }
}

In [15]:
// Display the values as a 2-D grid. Input: sudoku in dictionary form
function displayFull(values) {
  const width = 1 + Math.max(...boxes.map(s => values[s].length));
  const line = Array.from({ length: 3 }, () => '-'.repeat(width * 3)).join('+');
  for (let r of rows) {
    let rowOutput = '';
    for (let c of cols) {
      rowOutput += values[r + c].padStart(width, ' ') + (c === '3' || c === '6' ? '|' : '');
    }
    console.log(rowOutput);
    if (r === 'C' || r === 'F') {
      console.log(line);
    }
  }
}

In [16]:
// ---- Solving Algorithms ----

In [17]:
// Elimination strategy
// The eliminate() function will search boxes with a single digit and
// eliminate that digit from its peers.
function eliminate(values) {
  for (let box of Object.keys(values)) {
    if (values[box].length === 1) {
      const eliminationValue = values[box];
      for (let peer of peers[box]) {
        values[peer] = values[peer].replace(eliminationValue, '');
      }
    }
  }
  return values;
}

In [18]:
// Only choice strategy
// The function will go through all the units, and if there is a unit with a
// digit that only fits in one possible box, it will assign that digit to that
// box.
function onlyChoice(values) {
  for (let unit of unitlist) {
    for (let digit of '123456789') {
      const dplaces = unit.filter(box => values[box].includes(digit));
      if (dplaces.length === 1) {
        values[dplaces[0]] = digit;
      }
    }
  }
  return values;
}

In [19]:
// Reduce puzzle by constraint propagation
// Repeatedly apply the eliminate() and only_choice() functions until the puzzle
// is solved. This is called 'Constraint Propagation'
function reducePuzzle(values) {
  let stalled = false;
  while (!stalled) {
    const solvedValuesBefore = Object.values(values).filter(value => value.length === 1).length;
    eliminate(values);
    onlyChoice(values);
    const solvedValuesAfter = Object.values(values).filter(value => value.length === 1).length;
    stalled = solvedValuesBefore === solvedValuesAfter;
    // Sanity check for boxes with zero available values
    if (Object.values(values).some(value => value.length === 0)) {
      return false;
    }
  }
  return values;
}

In [20]:
// ---- Search ----
function search(values) {
  // First, reduce the puzzle using the previous function
  values = reducePuzzle(values);
  if (!values) {
    return false;
  }
  if (Object.values(values).every(value => value.length === 1)) {
    return values; // Solved!
  }
  // Using depth-first search and propagation, try all possible values.
  // Choose one of the unfilled squares with the fewest possibilities
  const [s, n] = Object.entries(values).filter(([s, value]) => value.length > 1).reduce((shortest, [s, value]) => value.length < shortest[1].length ? [s,  value] : shortest);

  // Now use recursion to solve each one of the resulting sudokus
  for (let value of values[s]) {
    const newSudoku = { ...values };  // Make a copy
    newSudoku[s] = value;
    const attempt = search(newSudoku);
    if (attempt) {
      return attempt;
    }
  }
}

In [21]:
// ---- Define Puzzles ----

In [22]:
const puzzle_1 = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

In [23]:
const puzzle_2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'

In [24]:
// Translate the puzzle string into an object mimicking an associative array (Python dict)
let puzzle_1_grid = gridValues(puzzle_1);
let puzzle_2_grid = gridValues(puzzle_2);

In [25]:
// ---- Solving Puzzle 1 ----

In [26]:
displayDot(puzzle_1_grid);

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


In [27]:
displayFull(puzzle_1_grid);

 123456789 123456789         3| 123456789         2 123456789|         6 123456789 123456789
         9 123456789 123456789|         3 123456789         5| 123456789 123456789         1
 123456789 123456789         1|         8 123456789         6|         4 123456789 123456789
------------------------------+------------------------------+------------------------------
 123456789 123456789         8|         1 123456789         2|         9 123456789 123456789
         7 123456789 123456789| 123456789 123456789 123456789| 123456789 123456789         8
 123456789 123456789         6|         7 123456789         8|         2 123456789 123456789
------------------------------+------------------------------+------------------------------
 123456789 123456789         2|         6 123456789         9|         5 123456789 123456789
         8 123456789 123456789|         2 123456789         3| 123456789 123456789         9
 123456789 123456789         5| 123456789         1 123456789|        

In [28]:
displayFull(search(puzzle_1_grid));

 4 8 3| 9 2 1| 6 5 7
 9 6 7| 3 4 5| 8 2 1
 2 5 1| 8 7 6| 4 9 3
------+------+------
 5 4 8| 1 3 2| 9 7 6
 7 2 9| 5 6 4| 1 3 8
 1 3 6| 7 9 8| 2 4 5
------+------+------
 3 7 2| 6 8 9| 5 1 4
 8 1 4| 2 5 3| 7 6 9
 6 9 5| 4 1 7| 3 8 2


In [29]:
// ---- Solving Puzzle 2 ----

In [30]:
displayDot(puzzle_2_grid);

4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 


In [31]:
displayFull(puzzle_2_grid);

         4 123456789 123456789| 123456789 123456789 123456789|         8 123456789         5
 123456789         3 123456789| 123456789 123456789 123456789| 123456789 123456789 123456789
 123456789 123456789 123456789|         7 123456789 123456789| 123456789 123456789 123456789
------------------------------+------------------------------+------------------------------
 123456789         2 123456789| 123456789 123456789 123456789| 123456789         6 123456789
 123456789 123456789 123456789| 123456789         8 123456789|         4 123456789 123456789
 123456789 123456789 123456789| 123456789         1 123456789| 123456789 123456789 123456789
------------------------------+------------------------------+------------------------------
 123456789 123456789 123456789|         6 123456789         3| 123456789         7 123456789
         5 123456789 123456789|         2 123456789 123456789| 123456789 123456789 123456789
         1 123456789         4| 123456789 123456789 123456789| 1234567

In [33]:
displayFull(search(puzzle_2_grid));

 4 1 7| 3 6 9| 8 2 5
 6 3 2| 1 5 8| 9 4 7
 9 5 8| 7 2 4| 3 1 6
------+------+------
 8 2 5| 4 3 7| 1 6 9
 7 9 1| 5 8 6| 4 3 2
 3 4 6| 9 1 2| 7 5 8
------+------+------
 2 8 9| 6 4 3| 5 7 1
 5 7 3| 2 9 1| 6 8 4
 1 6 4| 8 7 5| 2 9 3
